In [1]:
# import libraries
import yaml
from pathlib import Path
from neuralhydrology.utils.configutils import create_config_files
from neuralhydrology.nh_run_scheduler import schedule_runs
from neuralhydrology.evaluation.tuning import generate_run_dir_patterns
from neuralhydrology.evaluation.tuning import get_best_model_per_seed
from neuralhydrology.nh_run import eval_run
import pickle
import matplotlib.pyplot as plt

### Perform two stage hyperparametertuning

1. **Stage**: 
   - Use three repetitions of each hyperparameter setting with different random seeds for initializing the weights.
   - All models are run for 100 epochs using the Adam optimizer with:
     - Learning rate: 5e−3
     - Batch size: 256
   - During training, the model is validated after every 4 epochs on validation period data.

In [2]:
# Create grid for first stage
grid1 = {
    'hidden_size': [8, 16, 32, 64],  # Hidden size
    'output_dropout': [0.0, 0.2, 0.4, 0.5],  # Dropout rate on the head layer
    'seed': [123, 999, 321]  # Use three repetitions of each hyperparameter setting with different random seeds
}
# Create configs for all possible parameter combinations
create_config_files(base_config_path=Path('base_stage1.yml'), modify_dict=grid1, output_dir=Path('configs_stage1'))

Finished. Configs are stored in configs_stage1


In [3]:
# Train models for all configs from one call of the first stage
schedule_runs(mode='train', directory=Path('configs_stage1'), gpu_ids=[0], runs_per_gpu=2)

Starting run 1/48: python /home/sngrj0hn/GitHub/neuralhydrology/neuralhydrology/nh_run.py train --config-file /home/sngrj0hn/GitHub/neuralhydrology/camels_harz/single_basin_multi_forcing/DE5_margarethenklippe_grane/configs_stage1/config_23.yml --gpu 0
Starting run 2/48: python /home/sngrj0hn/GitHub/neuralhydrology/neuralhydrology/nh_run.py train --config-file /home/sngrj0hn/GitHub/neuralhydrology/camels_harz/single_basin_multi_forcing/DE5_margarethenklippe_grane/configs_stage1/config_32.yml --gpu 0
Finished run python /home/sngrj0hn/GitHub/neuralhydrology/neuralhydrology/nh_run.py train --config-file /home/sngrj0hn/GitHub/neuralhydrology/camels_harz/single_basin_multi_forcing/DE5_margarethenklippe_grane/configs_stage1/config_23.yml --gpu 0
Cleaning up...


Moved /home/sngrj0hn/GitHub/neuralhydrology/camels_harz/single_basin_multi_forcing/DE5_margarethenklippe_grane/configs_stage1/config_23.yml into directory of processed configs at configs_stage1/processed/config_23.yml.
Starting run 3

In [4]:
# Get run directories for the first stage
run_patterns = generate_run_dir_patterns(grid1, 'single_basin_multi_forcing')

In [5]:
# Get best model for each seed
best_models_per_seed = get_best_model_per_seed(parent_dir='runs', seeds=grid1['seed'], grid_params=list(grid1.keys())[:-1], run_patterns=run_patterns)

Evaluating repetition for seed 123.
Evaluating repetition for seed 999.
Evaluating repetition for seed 321.
Finished.


In [7]:
# Print epoch, median NSE, and hyperparameters for each seed
def print_grid_parameters(best_models_per_seed, grid_params):
    for seed, result in best_models_per_seed.items():
        if result['best_model_config'] is not None:
            config = result['best_model_config']
            params = {k: config.get(k) for k in grid_params if k in config}
            print(f"Seed: {seed}")
            print(f"Best epoch: {result['best_epoch']}")
            print(f"Best median NSE score: {result['best_nse']}")
            print(f"Parameters: {params}")
            print()
print_grid_parameters(best_models_per_seed, list(grid1.keys())[:-1])

Seed: 123
Best epoch: 12
Best median NSE score: 0.6085308194160461
Parameters: {'hidden_size': 64, 'output_dropout': 0.0}

Seed: 999
Best epoch: 24
Best median NSE score: 0.6269711852073669
Parameters: {'hidden_size': 64, 'output_dropout': 0.0}

Seed: 321
Best epoch: 19
Best median NSE score: 0.6211143732070923
Parameters: {'hidden_size': 32, 'output_dropout': 0.0}



In [ ]:
# Chose overall best model and hyperparameters based on median NSE for stage 1 and write parameters to a dictionary
best_model = None
best_nse = -float('inf')
best_seed = None
best_params1 = None

for seed, result in best_models_per_seed.items():
    if result['best_nse'] > best_nse:
        best_nse = result['best_nse']
        best_model = result['best_model_dir']
        # best params excluding the seed
        best_params1 = {k: v for k, v in result['best_model_config'].items() if k in grid1}

2. **Stage**: 
   - Using the hyperparameters chosen from the first stage, we tune the learning rate and batch size  in a similar way, maximizing over the median NSE over 3 model repetitions.
   - All models are run again for 100 epochs using the Adam optimizer.
   - During training, the model is validated after every 4 epochs on validation period data.

In [ ]:
# Create config file for the second stage including hyperparameters of the best model from the first stage
# Read in the base.yml file and replace the hyperparameters with the best hyperparameters from the first stage
with open('base_stage1.yml', 'r') as file:
    base_config = yaml.safe_load(file)

# Update the base config with the best hyperparameters from the first stage
base_config.update(best_params1)

# Save updated config to a new file
with open('base_stage2.yml', 'w') as file:
    yaml.dump(base_config, file)

# Create grid for second stage
grid2 = {
    'learning_rate': [0.01, 0.001, 0.0001],  # Learning rate
    'batch_size': [32, 64, 128],  # Batch size
    'seed': [123, 999, 321]  # Use three repetitions of each hyperparameter setting with different random seeds
}

create_config_files(base_config_path=Path('base_stage2.yml'), modify_dict=grid2, output_dir=Path('configs_stage2'))

In [ ]:
# Train models for all configs from one call of stage 2
schedule_runs(mode='train', directory=Path('configs_stage2'), gpu_ids=[0], runs_per_gpu=2)

In [ ]:
# Get run directories for the second stage
run_patterns = generate_run_dir_patterns(grid2, 'single_basin_multi_forcing')

In [ ]:
# Get best model for each seed
best_models_per_seed = get_best_model_per_seed(parent_dir='runs', seeds=grid2['seed'], grid_params=list(grid2.keys())[:-1], run_patterns=run_patterns)

In [ ]:
# Print epoch, median NSE, and hyperparameters for each seed
print_grid_parameters(best_models_per_seed, list(grid2.keys())[:-1])

In [ ]:
# Chose overall best model and hyperparameters based on median NSE for stage 2 and write parameters to a dictionary
best_model = None
best_nse = -float('inf')
best_seed = None
best_params2 = None

for seed, result in best_models_per_seed.items():
    if result['best_nse'] > best_nse:
        best_nse = result['best_nse']
        best_model = result['best_model_dir']
        best_epoch = f"{result['best_epoch']}"
        # best params excluding the seed
        best_params2 = {k: v for k, v in result['best_model_config'].items() if k in grid2}

In [ ]:
# Create config file for the second stage including hyperparameters of the best model from the first stage
# Read in the base.yml file and replace the hyperparameters with the best hyperparameters from the first stage
with open('base_stage2.yml', 'r') as file:
    base_config = yaml.safe_load(file)

# Update the base config with the best hyperparameters from the first stage
base_config.update(best_params2)

# Save the updated config to a new file
with open('base_final.yml', 'w') as file:
    yaml.dump(base_config, file)

### Evaluate run on test set
The path to the best model is automatically saved in the variable best_model, and the corresponding best epoch is stored as a string in best_epoch.

In [ ]:
eval_run(run_dir=best_model, period="test", epoch=best_epoch)

### Load and inspect model predictions
Load the results file and compare the model predictions with observations. The results file is always a pickled dictionary with one key per basin (even for a single basin). The next-lower dictionary level is the temporal resolution of the predictions. In this case, we trained a model only on daily data ('1D'). Within the temporal resolution, the next-lower dictionary level are `xr`(an xarray Dataset that contains observations and predictions), as well as one key for each metric that was specified in the config file.

In [ ]:
with open(best_model / "test" / f"model_epoch0{best_epoch}" / "test_results.p", "rb") as fp:
    results = pickle.load(fp)
    
results.keys()

In [ ]:
results['DE4']['1D']['xr']

In [ ]:
# Extract observations and simulations
qobs = results['DE4']['1D']['xr']['discharge_vol_obs']
qsim = results['DE4']['1D']['xr']['discharge_vol_sim']

fig, ax = plt.subplots(figsize=(16,10))
ax.plot(qobs['date'], qobs)
ax.plot(qsim['date'], qsim)
ax.set_ylabel("Discharge (m³/s)")
ax.set_title(f"Test period - NSE {results['DE4']['1D']['NSE']:.3f}")